# StateGraph in LangGraph: The Core Graph Builder

StateGraph is the fundamental class in LangGraph that allows you to build and manage stateful graphs. It provides the infrastructure for creating complex workflows where state flows between nodes, enabling sophisticated AI applications with memory and context.

## Key Features:
- **State Management**: Handles state flow between nodes automatically
- **Node Orchestration**: Manages execution order and dependencies
- **Edge Control**: Defines how data flows through the graph
- **Compilation**: Converts graph definition into executable application

## Creating a StateGraph:

### **Basic StateGraph Creation**
```python
from langgraph import StateGraph
from typing import TypedDict

# Define state structure
class MyState(TypedDict):
    user_input: str
    processed_data: str
    result: str

# Create StateGraph
graph = StateGraph(MyState)
```

### **StateGraph with Custom State**
```python
class ChatState(TypedDict):
    messages: list[str]
    user_id: str
    session_id: str
    context: dict
    response: str

# Create graph with custom state
chat_graph = StateGraph(ChatState)
```

## Building the Graph:

### **Adding Nodes**
```python
def process_input(state: MyState) -> dict:
    """Process user input"""
    return {"processed_data": state["user_input"].upper()}

def generate_response(state: MyState) -> dict:
    """Generate response based on processed data"""
    return {"result": f"Processed: {state['processed_data']}"}

# Add nodes to graph
graph.add_node("process", process_input)
graph.add_node("generate", generate_response)
```

### **Adding Edges**
```python
# Simple edge
graph.add_edge("process", "generate")

# Set entry point
graph.set_entry_point("process")
```

### **Compiling the Graph**
```python
# Compile graph into executable application
app = graph.compile()

# Execute the graph
result = app.invoke({"user_input": "Hello World"})
```

## Advanced Graph Patterns:

### **Conditional Edges**
```python
def route_decision(state: MyState) -> str:
    """Decide next step based on state"""
    if state.get("processed_data", "").startswith("ERROR"):
        return "error_handler"
    else:
        return "success_handler"

# Add conditional routing
graph.add_conditional_edges(
    "process",
    route_decision,
    {
        "error_handler": "handle_error",
        "success_handler": "generate"
    }
)
```

### **Multiple Entry Points**
```python
# Multiple possible starting points
graph.add_node("start_a", process_a)
graph.add_node("start_b", process_b)

# Set multiple entry points
graph.set_entry_point("start_a")
# Note: Only one entry point per graph, but you can use conditional logic
```

### **Complex Graph Structure**
```python
# Create a complex workflow
graph.add_node("classify", classify_input)
graph.add_node("weather", handle_weather)
graph.add_node("news", handle_news)
graph.add_node("general", handle_general)
graph.add_node("format", format_response)

# Add conditional routing
graph.add_conditional_edges(
    "classify",
    lambda state: state["query_type"],
    {
        "weather": "weather",
        "news": "news",
        "general": "general"
    }
)

# All paths lead to formatter
graph.add_edge("weather", "format")
graph.add_edge("news", "format")
graph.add_edge("general", "format")
```

## State Management:

### **State Updates**
```python
def update_state(state: MyState) -> dict:
    """Update state with new information"""
    return {
        "processed_data": state["user_input"].strip(),
        "timestamp": datetime.now().isoformat(),
        "step_count": state.get("step_count", 0) + 1
    }
```

### **State Validation**
```python
def validate_state(state: MyState) -> dict:
    """Validate state before processing"""
    if not state.get("user_input"):
        return {"error": "No input provided", "status": "error"}
    
    if len(state["user_input"]) < 3:
        return {"error": "Input too short", "status": "error"}
    
    return {"status": "valid", "validated": True}
```

### **State Transformation**
```python
def transform_state(state: MyState) -> dict:
    """Transform state for next step"""
    return {
        "processed_data": state["user_input"].lower(),
        "word_count": len(state["user_input"].split()),
        "char_count": len(state["user_input"])
    }
```

## Graph Execution:

### **Synchronous Execution**
```python
# Execute graph synchronously
result = app.invoke({
    "user_input": "Hello World",
    "processed_data": "",
    "result": ""
})

print(result["result"])  # Output: Processed: HELLO WORLD
```

### **Asynchronous Execution**
```python
import asyncio

async def run_graph():
    result = await app.ainvoke({
        "user_input": "Hello World",
        "processed_data": "",
        "result": ""
    })
    return result

# Run async
result = asyncio.run(run_graph())
```

### **Streaming Execution**
```python
# Stream intermediate results
for chunk in app.stream({"user_input": "Hello World"}):
    print(f"Step: {chunk}")
```

## Graph Configuration:

### **Graph Metadata**
```python
# Create graph with metadata
graph = StateGraph(
    MyState,
    name="MyWorkflow",
    description="A sample workflow graph"
)
```

### **Node Configuration**
```python
# Add node with configuration
graph.add_node(
    "process",
    process_input,
    name="Input Processor",
    description="Processes user input"
)
```

### **Edge Configuration**
```python
# Add edge with metadata
graph.add_edge(
    "process",
    "generate",
    name="Process to Generate",
    description="Flow from processing to generation"
)
```

## Error Handling:

### **Node Error Handling**
```python
def safe_node(state: MyState) -> dict:
    """Node with error handling"""
    try:
        result = risky_operation(state["user_input"])
        return {"result": result, "status": "success"}
    except Exception as e:
        return {"error": str(e), "status": "error"}
```

### **Graph-Level Error Handling**
```python
def error_handler(state: MyState) -> dict:
    """Handle errors at graph level"""
    return {
        "error_message": state.get("error", "Unknown error"),
        "status": "failed",
        "recovery_suggestion": "Please try again"
    }

# Add error handling node
graph.add_node("error_handler", error_handler)
```

## Graph Visualization:

### **Graph Structure**
```python
# Get graph structure
print("Nodes:", graph.nodes)
print("Edges:", graph.edges)
print("Entry point:", graph.entry_point)
```

### **Graph Validation**
```python
# Validate graph before compilation
try:
    app = graph.compile()
    print("Graph compiled successfully")
except Exception as e:
    print(f"Graph compilation failed: {e}")
```

## Best Practices:

### **1. State Design**
- Keep state structure simple and focused
- Use descriptive field names
- Avoid deeply nested state
- Consider state immutability

### **2. Node Design**
- Make nodes focused and single-purpose
- Handle errors gracefully
- Keep nodes stateless when possible
- Document node behavior

### **3. Edge Design**
- Use clear, logical flow
- Implement proper error routing
- Consider all possible paths
- Test edge conditions

### **4. Graph Organization**
- Group related nodes together
- Use descriptive names
- Document graph structure
- Keep graphs focused on specific workflows

## Common Patterns:

### **Linear Workflow**
```python
# Simple sequential processing
graph.add_edge("start", "process")
graph.add_edge("process", "generate")
graph.add_edge("generate", "end")
```

### **Branching Workflow**
```python
# Conditional branching
graph.add_conditional_edges(
    "classifier",
    lambda state: state["type"],
    {
        "type_a": "handler_a",
        "type_b": "handler_b"
    }
)
```

### **Converging Workflow**
```python
# Multiple paths converge
graph.add_edge("handler_a", "merger")
graph.add_edge("handler_b", "merger")
graph.add_edge("merger", "finalizer")
```

## Benefits:
- **Stateful Processing**: Maintain context across nodes
- **Flexible Architecture**: Support complex workflows
- **Error Handling**: Built-in error management
- **Scalability**: Handle large, complex applications
- **Maintainability**: Clear separation of concerns

## Tips for Success:
- Start with simple graphs and add complexity
- Use descriptive names for nodes and edges
- Handle errors at every level
- Test with various state inputs
- Document your graph structure
- Consider performance implications
- Plan for state evolution
- Use type hints for better development experience
